# Exploring ACTS 
## Lara Calic, Hannah Herde 
### Tuesday 13 December 2022 

### Objective 
We are trying to understand the code blocks in `full_detector_odd.py`

--- 

### Set up our `imports`

In [1]:
import pathlib, acts, acts.examples

In [2]:
from acts.examples.simulation import (
    addParticleGun,
    MomentumConfig,
    EtaConfig,
    ParticleConfig,
    addPythia8,
    addFatras,
    ParticleSelectorConfig,
    addDigitization,
)

In [3]:
from acts.examples.reconstruction import (
    addSeeding,
    TruthSeedRanges,
    addCKFTracks,
    CKFPerformanceConfig,
    TrackSelectorRanges,
    addAmbiguityResolution,
    AmbiguityResolutionConfig,
    addVertexFitting,
    VertexFinder,
)

In [4]:
# Manipute system with sys, add modules to path 
import sys 
sys.path.append( '/my-acts/source/Examples/Scripts/Python/' )

from common import getOpenDataDetectorDirectory

Ah, wait, we see that this function from `common.py` makes a path relative to our run directory. Yikes :( We hardcoded the path in `common.py` 

Let's test it

In [5]:
getOpenDataDetectorDirectory()

PosixPath('/my-acts/source/thirdparty/OpenDataDetector')

In [6]:
from acts.examples.odd import getOpenDataDetector

## Keep in mind: 
OK, let's posit that we run our script in a different directory than `Examples/Scripts/Python/`. The examples seem to reference everything based on the python examples directory :(. **We would need to replace all these paths, or RUN from that examples directory.** We fixed this by hard-coding the corrected path in `common`; there may be others! 

We need to take a decision: Are we going to update these paths?

### Moving past the imports... 
Setting up the constants from Acts::UnitConstants from the C++ core libraries 

In [7]:
u = acts.UnitConstants

1. Establish path for the geometry of the Open Data Detector (ODD) 
2. Give an output directory 

In [8]:
geoDir = getOpenDataDetectorDirectory()
outputDir = pathlib.Path.cwd() / "odd_output"
print( geoDir, outputDir )

/my-acts/source/thirdparty/OpenDataDetector /Users/hherde/cernbox/Research/ATLAS/CombinedPerformance/Tracking/my-acts/run/odd_output


Show the python binding calls 

In [9]:
acts.examples.dump_args_calls(locals()) 

dump_args for module functions: acts:53, acts.examples:257, acts.examples.simulation:6, acts.ActsPythonBindings:34, acts.ActsPythonBindings.logging:21, acts.ActsPythonBindings._examples:113, acts.ActsPythonBindings._examples.pythia8:2, acts.ActsPythonBindings._examples._edm4hep:14, acts.ActsPythonBindings._examples._hepmc3:6, acts.ActsPythonBindings._propagator:6


512

From the geometry directory, specify the material map, digital configuration, track seeding 

In [10]:
oddMaterialMap = geoDir / "data/odd-material-maps.root"
oddDigiConfig = geoDir / "config/odd-digi-smearing-config.json"
oddSeedingSel = geoDir / "config/odd-seeding-config.json"

In [11]:
# Test that I can open the json files 
with open( oddSeedingSel, 'r') as f:
    print(f.read())

[
    {
        "layer": 2,
        "volume": 17
    },
    {
        "layer": 4,
        "volume": 17
    },
    {
        "layer": 6,
        "volume": 17
    },
    {
        "layer": 8,
        "volume": 17
    },
    {
        "layer": 2,
        "volume": 18
    },
    {
        "layer": 4,
        "volume": 18
    },
    {
        "layer": 6,
        "volume": 18
    },
    {
        "layer": 8,
        "volume": 18
    },
    {
        "layer": 16,
        "volume": 16
    },
    {
        "layer": 14,
        "volume": 16
    },
    {
        "layer": 12,
        "volume": 16
    },
    {
        "layer": 10,
        "volume": 16
    }
]



Open the material ROOT file with ACTS 

In [12]:
oddMaterialDeco = acts.IMaterialDecorator.fromFile(oddMaterialMap)

acts.ActsPythonBindings._examples.RootMaterialDecorator ( fileName = '/my-acts/source/thirdparty/OpenDataDetector/data/odd-material-maps.root', level = <Level.INFO: 2> )


### Excellent! Further than ever before!! 
We have the material map in the correct directory now and apparently can reference it properly. That's great! 

Now, we want to fetch the Open Data Detector (ODD) with ACTS as an object with the supplied material smearing and seeding maps. 

In [13]:
import acts.examples.dd4hep
detector, trackingGeometry, decorators = getOpenDataDetector(
    geoDir, mdecorator=oddMaterialDeco
)

RuntimeError: Unable to find OpenDataDetector factory library. You might need to point LD_LIBRARY_PATH to build/thirdparty/OpenDataDetector/factory or other ODD install location